In [92]:
import sqlite3
import os
import shutil
import datetime
import re

In [93]:
# db_name = 'blank.db'

# current_time = datetime.datetime.now().strftime('%H-%M-%S')
    
#     # Get the file extension
# file_extension = db_name.split('.')[-1]

# new_file_name = f"{current_time}.{file_extension}"

# # Create new file path
# new_file_path = f"old/{new_file_name}"

# # Copy and rename the file
# shutil.copy2(db_name, new_file_path)

In [94]:
blank_name = 'blank.db'
excel_name = 'cs202109'
db_name = 'parsed_data/' + excel_name + '.db'
if os.path.isfile(db_name):
    os.remove(db_name)
shutil.copy2(blank_name, db_name)

'parsed_data/cs202109.db'

In [95]:
#connect
conn = sqlite3.connect(db_name)
c = conn.cursor()

In [96]:
#create tables

# c.execute("DROP TABLE IF EXISTS descriptions;")
# c.execute("DROP TABLE IF EXISTS data;")
# c.execute("""CREATE TABLE descriptions (
#     id INTEGER PRIMARY KEY,
#     description TEXT,
#     x_unit TEXT,
#     y_unit TEXT
# );""")
# c.execute("""CREATE TABLE data (
#     id INTEGER PRIMARY KEY,
#     description_id INTEGER,
#     x_text TEXT,
#     x_value INTEGER,
#     x_datetime DATETIME,
#     value REAL,
#     name TEXT,
#     FOREIGN KEY(description_id) REFERENCES descriptions(id)
# );""")

In [97]:
#delete all
# c.execute("DELETE FROM descriptions")
# c.execute("DELETE FROM data")

In [98]:
current_desc_id = 1
c.execute(f"SELECT * FROM descriptions WHERE id = (SELECT MAX(id) FROM descriptions);")
result = c.fetchall()
if result:
    current_desc_id = result[0][0] + 1
print(current_desc_id)

1


In [99]:
current_data_id = 1
c.execute(f"SELECT * FROM data WHERE id = (SELECT MAX(id) FROM data);")
result = c.fetchall()
if result:
    current_data_id = result[0][0] + 1
print(current_data_id)

1


In [100]:
def insert_description(description, x_unit, y_unit):
    c.execute("SELECT id FROM descriptions WHERE description = ?", (description,))
    result = c.fetchone()
    if result:
        return result[0]
    global current_desc_id
    # Insert a row of data into the 'descriptions' table
    c.execute(f"INSERT INTO descriptions VALUES ({current_desc_id}, '{description}', '{x_unit}', '{y_unit}')")
    current_desc_id += 1
    return current_desc_id - 1

def insert_data(desc_id, x_text, x_value, x_datetime, value, name):
    global current_data_id
    # Insert a row of data into the 'data' table
    c.execute("INSERT INTO data VALUES (?, ?, ?, ?, ?, ?, ?)", 
          (current_data_id, desc_id, x_text, x_value, x_datetime, value, name))
    current_data_id += 1
    return current_data_id - 1

In [101]:
def extract_number(s):
    match = re.search(r'\d+', s)
    return int(match.group()) if match else None

In [102]:
input_string = """
CSNg ar cietušajiem	4056	3711	3925	4540	4442	4482	4766	5083	5379	5081	4466	4302	4781	4196	3160	3193	3386	3358	3489	3728	3692	3792	3875	3975	3729	3403
	947	768	850	963	916	822	701	775	691	633	547	435	519	403	258	232	270	174	211	210	196	187	166	199	173	201
Bojā gājuši (30 dienas)	660	594	567	677	652	635	558	559	532	516	442	407	419	316	254	218	179	177	179	212	188	158	136	148	132	139
	203	139	170	183	183	125	111	160	119	113	96	84	91	58	36	22	26	25	10	29	18	17	12	11	14	7
Ievainoti	4854	4280	4632	5364	5196	5402	5811	6259	6600	6416	5600	5404	6088	5408	3930	4023	4224	4179	4338	4603	4566	4648	4824	4798	4553	4059
	1294	1066	1142	1282	1234	1228	989	1088	955	940	821	655	780	565	372	317	347	277	302	296	280	259	244	272	232	263
"""


In [103]:
# # cs202109.xlsx

# years = ["1995", "1996", "1997", "1998", "1999", "2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"]

# lines = input_string.splitlines()
# alk = False
# description = ""
# description_place = -1
# description_id = -1

# for line in lines:
#     if line == "":
#         continue
#     words = line.split()
#     if not alk:
#         for i, word in enumerate(words):
#             if word.isdigit():
#                 description_place = i
#                 break
#         description = " ".join(words[:description_place])
#         description_id = insert_description(description, "gads", "skaits")
#     else:
#         description += " vadītājs bija alkohola reibumā"
#         description_id = insert_description(description, "gads", "skaits")
#     data = words[description_place:]
#     for i, data_i in enumerate(data):
#         insert_data(description_id, years[i], int(years[i]), datetime.date(int(years[i]), 1, 1), int(data_i), "")
#     alk = not alk
        


In [104]:
# # cs202108.xlsx
# years = ["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"]


# lines = input_string.splitlines()
# page = "CSNg ar bojā ievainotajiem Rīgā"
# vehicle_count = ""
# for line in lines:
#     if line.startswith("Divu") or line.startswith("Viena"):
#         vehicle_count = line
#         continue
#     description = ""
#     description_id = -1
#     words = line.split()
#     year_counter = 0
#     for i, word in enumerate(words):
#         if word.isdigit():
#             if not description:
#                 description = " ".join(words[:i])
#                 if vehicle_count:
#                     full_description = f"{page} - {vehicle_count} - {description},"
#                 else:
#                     full_description = f"{page} - {description},"
#                 description_id = insert_description(full_description, "gads", "skaits")
#             insert_data(description_id, years[year_counter], int(years[year_counter]), datetime.date(int(years[year_counter]), 1,1), int(word), "")
#             year_counter += 1


In [105]:
# #cs202107.xlsx
# columns = ["gads",	"CSNg ar cietušajiem",	"Bojā gājuši (30 dienas)",	"Ievainoti (kopā)",	"Smagi ievainoti"]			


# months = ["Janvāris", "Februāris", "Marts", "Aprīlis", "Maijs", "Jūnijs", "Jūlijs", "Augusts", "Septembris", "Oktobris", "Novembris", "Decembris" ]
# lines = input_string.splitlines()

# years = []

# year_counter = 1
# description_counter = 1
# alk = False

# year_index = -1

# description = ""
# alk_description = ""

# description_id = -1
# alk_description_id = -1

# year = ""
# year_int = -1
# year_datetime = None

# years_done = []

# lines_clean = []


# for line in lines:
#     line = line.replace('\t', ' ')
#     if line.isspace() or not line:
#         continue
#     else:
#         lines_clean.append(line)

# for line in lines_clean:
#     words = line.split()

#     if year_counter == 3:   #every third line has a year
#         year_counter = 1
#     if year_counter == 1:
#         if not words[0][0].isdigit():   #if first is a description
#             for i, word in enumerate(words):    #find the year
#                 if word[0].isdigit():
#                     year_index = i
#                     break
#         else:
#             year_index = 0  #else, year is the first word
#         year = words[year_index]    #extract info about the year
#         year_int = extract_number(year)
#         year_datetime = datetime.date(year_int, 1, 1)

#     if description_counter == 5: #every fith line has a description
#         description_counter = 1
#     if description_counter == 1:    
#         description = ' '.join(words[:year_index])
#         description_id = insert_description(description, x_unit="menesis", y_unit="skaits")
#         alk_description = description + " alkohola ietekmē"
#         alk_description_id = insert_description(alk_description, x_unit="menesis", y_unit="skaits")

#     cursor = 0
#     if year_counter == 1:
#         cursor = year_index + 1
#     year_enum = enumerate(words[cursor:])
#     for i, word in year_enum:
#         month = months[i]
#         if alk:
#             insert_data(alk_description_id, f"{month} {year}", month, datetime.date(year_int, i + 1, 1), word, "")
#         else:
#             insert_data(description_id, f"{month} {year}", month, datetime.date(year_int, i + 1, 1), word, "")
#     year_counter += 1
#     description_counter += 1
#     alk = not alk
    

In [106]:
# #cs202103.xlsx

# dates_unprocessed = date_string.split()
# dates = []
# for date in dates_unprocessed:
#     possible_date = extract_number(date)
#     if possible_date is not None:
#         dates.append(possible_date)

# lines = input_string.splitlines()

# for line in lines:
#     if not("T.sk." in line or "no tiem" in line):
#         description = ""
#     words = line.split()
#     description_id = 0
#     date_cursor = 0
#     data = []
#     for word in words:
#         if word.isdigit():  
#             if description_id == 0:
#                 description_id = insert_description(description, "gads", "skaits")
#             year = dates[date_cursor]
#             year_int = year
#             year_datetime = datetime.date(year_int, 1, 1)
#             insert_data(description_id, str(year), year_int, year_datetime, word, "")
#             date_cursor += 1
#         else:
#             description += word
#             description += " "

In [107]:
# #cs202104.xlsxl
# column_ids = [0]
# for column in columns:
#     if column != "gads":
#         id = insert_description(column, "gads", "skaits")
#         column_ids.append(id)
# lines = input_string.splitlines()
# for line in lines:
#     words = line.split()
#     year = 0
#     year_int = 0
#     year_datetime = datetime.date(1, 1, 1)
#     for word, col_id in zip(words, column_ids):
#         if year == 0:
#             year = word
#             year_int = extract_number(word)
#             year_datetime = datetime.date(year_int, 1, 1)
#         else:
#             insert_data(col_id, year, year_int, year_datetime, word, "")

In [108]:
# Save (commit) the changes
conn.commit()

# c.execute("SELECT name FROM sqlite_master WHERE type='table';")
# tables = c.fetchall()

# # Loop through all tables and print out their contents
# for table in tables:
#     print(f"Table: {table[0]}")
#     c.execute(f"SELECT * FROM {table[0]};")
#     rows = c.fetchall()
#     for row in rows:
#         print(row)

In [109]:
# Close the connection
conn.close()